In [32]:
from adjusted_rand_index import rand_index
import os
import io
import math
from PIL import Image, ImageDraw
from threading import Thread
import numpy as np
from subprocess import call

DATASET_FOLDER = 'videos/'
TEST_VIDEO = 'ZZTM0X6OON1U.mp4'

In [35]:
def get_keyframes_from_video(video):
    print("Getting keyframes from video")
    test_path = DATASET_FOLDER + "/" + TEST_VIDEO
    call("""ffmpeg -i """ + DATASET_FOLDER + video + """ -vf "select='eq(pict_type,PICT_TYPE_I)'" -vsync vfr images/image_%d.png""", shell=True)

In [41]:
def delete_images():
    call("rm images/*.png", shell=True)

In [ ]:
def LSH_frame(image):
    

In [37]:
get_keyframes_from_video(TEST_VIDEO)

Getting keyframes from video


In [40]:
delete_images()

In [27]:
# Work flow:
# Open video file
# Extract frame from video (Extract x number of frames from video?)
# Hash frame
# Compare hash value with other hash values
# Cluster videos based on similarity of hash values